# Stock Data Visualization Dashboard with Bokeh

## Getting stock data from pandas_datareader

In [2]:
from pandas_datareader import data
import datetime

In [3]:
# pandas_datareader doc: https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#remote-data-yahoo
#AAPL: stock ticker for Apple

start=datetime.datetime(2021,12,20)
end=datetime.datetime(2021,12,27)
df1 = data.DataReader(name="AAPL", data_source="yahoo",start=start,end=end)
df1

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-12-20,170.580002,167.460007,168.279999,169.750000,107499100,169.750000
2021-12-21,173.199997,169.119995,171.559998,172.990005,91185900,172.990005
2021-12-22,175.860001,172.149994,173.039993,175.639999,92135300,175.639999
2021-12-23,176.850006,175.270004,175.850006,176.279999,68227500,176.279999
2021-12-27,180.419998,177.070007,177.089996,180.330002,74717300,180.330002


In [4]:
#GOOG: stock ticker for Google

start=datetime.datetime(2021,12,20)
end=datetime.datetime(2021,12,27)
df2 = data.DataReader(name="GOOG", data_source="yahoo",start=start,end=end)
df2

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-12-20,2852.209961,2805.000000,2813.592041,2848.030029,1013200,2848.030029
2021-12-21,2893.841064,2834.699951,2863.000000,2884.409912,977400,2884.409912
2021-12-22,2946.060059,2879.260010,2882.000000,2938.979980,921900,2938.979980
2021-12-23,2971.451904,2939.017090,2941.790039,2942.850098,690300,2942.850098
2021-12-27,2968.530029,2945.000000,2949.270020,2961.280029,662800,2961.280029


## Building Candlestick Chart

In [36]:
from bokeh.plotting import figure, show, output_file

start=datetime.datetime(2020,12,15)
end=datetime.datetime(2021,12,27)
df=data.DataReader(name="GOOG", data_source="yahoo",start=start,end=end)

In [37]:
#we generate a new column "Status"
def inc_dec(c,o):
    if c > o:
        value="Increase"
    elif c < o:
        value="Decrease"
    else:
        value="Equal"
    return value

df["Status"]=[inc_dec(c,o) for c,o in zip(df.Close,df.Open)]

#we generate a new column Middle with the avg between the open and close values
df["Middle"] = (df.Open+df.Close)/2
df["Height"] = abs(df.Open-df.Close)

In [38]:
df.tail(10)

,High,Low,Open,Close,Volume,Adj Close,Status,Middle,Height
Date,,,,,,,,,
2021-12-13,2971.250000,2927.199951,2968.879883,2934.090088,1205200,2934.090088,Decrease,2951.484985,34.789795
2021-12-14,2908.840088,2844.850098,2895.399902,2899.409912,1238900,2899.409912,Increase,2897.404907,4.010010
2021-12-15,2950.344971,2854.110107,2887.320068,2947.370117,1364000,2947.370117,Increase,2917.345093,60.050049
2021-12-16,2971.030029,2881.850098,2961.540039,2896.770020,1370000,2896.770020,Decrease,2929.155029,64.770020
2021-12-17,2889.201904,2835.760010,2854.290039,2856.060059,2162800,2856.060059,Increase,2855.175049,1.770020
2021-12-20,2852.209961,2805.000000,2813.592041,2848.030029,1013200,2848.030029,Increase,2830.811035,34.437988
2021-12-21,2893.841064,2834.699951,2863.000000,2884.409912,977400,2884.409912,Increase,2873.704956,21.409912
2021-12-22,2946.060059,2879.260010,2882.000000,2938.979980,921900,2938.979980,Increase,2910.489990,56.979980
2021-12-23,2971.451904,2939.017090,2941.790039,2942.850098,690300,2942.850098,Increase,2942.320068,1.060059


In [55]:
p = figure(x_axis_type='datetime', width=1000, height=300, sizing_mode="scale_width")
p.title="Candlestick Chart"
p.grid.grid_line_alpha=0.3 #level of transparency of grid

#rectangles parameters:
# x_center
# y_center
# width in ms
# height

hours_12 = 12*60*60*1000

#we first create the segments showing the highest and lowest values
p.segment(df.index, df.High, df.index, df.Low, color="Black")

#we now create green rectangles corresponding to dates where
#the Close values were higher than the Open values

p.rect(df.index[df.Status=="Increase"],df.Middle[df.Status=="Increase"],hours_12,
       df.Height[df.Status=="Increase"],fill_color="#90EE90",line_color="black")

#we then create red rectangles corresponding to dates where
#the Close values were lower than the Open values

p.rect(df.index[df.Status=="Decrease"],df.Middle[df.Status=="Decrease"],hours_12,
       df.Height[df.Status=="Decrease"],fill_color="#F08080",line_color="black")



output_file("CS.html")
show(p)

## Embedding Bokeh in a Webpage

In [45]:
from bokeh.embed import components
from bokeh.resources import CDN #content delivery network

In [52]:
#script1 contains the javascript code
#div1 contains the html

script1, div1 = components(p)
cdn_js=CDN.js_files

In [53]:
cdn_js[0]

'https://cdn.bokeh.org/bokeh/release/bokeh-2.4.1.min.js'